# Spark Streaming
This notebook takes in the `gemini-feed` Kafka topic and produces to the `spark.out` topic a feed which includes the order price volume ratio and bid/ask liquidity for BTC.

In [1]:
!pip install kafka-python

In [2]:
import os  
os.environ['PYSPARK_SUBMIT_ARGS'] =  '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.3 pyspark-shell'

In [3]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils  
from kafka import KafkaProducer
import json
import time

In [4]:
producer = KafkaProducer(bootstrap_servers='kafka-node:9092')

In [5]:
sc = SparkContext(appName="PythonSparkStreamingKafka_RM_01")  
sc.setLogLevel("ERROR") 
ssc = StreamingContext(sc, 60)  

num_streams=5 
kafkaStreams = [KafkaUtils.createStream(ssc,"kafka-node:2181","spark-streaming",{"gemini-feed":1}) for i in range(num_streams)]
unified_stream = ssc.union(*kafkaStreams)


In [6]:
kafkaStream = KafkaUtils.createStream(ssc, 'kafka-node:2181', 'spark-streaming', {'gemini-feed':1})  

In [7]:
parsed = kafkaStream.map(lambda v: json.loads(v[1]))

In [8]:
def handler(message):
    records = message.collect()
    for record in records:
        output = {}
        output['type'] = record[0]
        output['value'] = record[1]
        output['timestamp'] = int(time.time())
        producer.send('spark.out', bytes(json.dumps(output), 'utf-8'))
        producer.flush()

In [9]:
def price_volume(x):
    price = float(x['price'])
    remaining = float(x['remaining'])
    x['price_volume'] = price * remaining
    return (x['side'], x['price_volume'])

Get price/volume ratio

In [10]:
parsed_pv = parsed.map(lambda x: price_volume(x))

Group by side (bid or ask)

In [11]:
grouped = parsed_pv.reduceByKey(lambda accum, n: accum + n)

In [12]:
grouped_sorted = grouped.transform(lambda rdd: rdd.sortByKey())

In [13]:
grouped_sorted.foreachRDD(handler)

Generate a ratio of the price_volume of ask to bids. So, if ask volume is 4000 and bid volume is 1000 then the ratio is 4. This suggests ask (sell) side liquidity greatly exceeds buy side.

In [14]:
results = grouped_sorted.map(lambda x: ('price_volume', x[1]))

In [15]:
ratio = results.reduceByKey(lambda x, y: x/y)

In [16]:
ratio.foreachRDD(handler)

Print results

In [17]:
grouped_sorted.pprint()
ratio.pprint()

In [18]:
ssc.start()

-------------------------------------------
Time: 2019-05-23 13:48:00
-------------------------------------------
('ask', 1045731.8959677313)
('bid', 2126749.3346212585)

-------------------------------------------
Time: 2019-05-23 13:48:00
-------------------------------------------
('price_volume', 0.49170434848353206)

-------------------------------------------
Time: 2019-05-23 13:49:00
-------------------------------------------
('ask', 4929771.5676380005)
('bid', 3054978.8419712596)

-------------------------------------------
Time: 2019-05-23 13:49:00
-------------------------------------------
('price_volume', 1.613684356798036)

-------------------------------------------
Time: 2019-05-23 13:50:00
-------------------------------------------
('ask', 10632722.82202867)
('bid', 10309057.078794688)

-------------------------------------------
Time: 2019-05-23 13:50:00
-------------------------------------------
('price_volume', 1.0313962509626364)

--------------------------------

-------------------------------------------
Time: 2019-05-23 14:16:00
-------------------------------------------

-------------------------------------------
Time: 2019-05-23 14:16:00
-------------------------------------------



In [25]:
ssc.stop()